# Correttezza dell'algoritmo perceptron

Un training set $D$ è composto da $n$ coppie $x^{(i)}, y^{(i)}$ dove gli $x^{(i)}$ sono punti in $R^d$ e $y^{(i)}$ sono etichette in $\{-1, 1\}$. Questo è separabile linearmente se esistono $a, a_0$ tale che per ogni punto  $x^{(i)}$

$$
y^{(i)}(a\cdot x^{(i)} + a_0) > 0
$$

ovvero tutte le previsioni sul training set sono corrette.

Dimostreremo che se l'insieme di punti è saparabile allora, elminando il parametro $t$ (ovvero con $t$ illimitato), l'algoritmo trova un piano di separazione.

### Equivalenza tra problemi con offset ($a_0$) e problemi senza

Sia $(a_1,...,a_d)$ con offset $a_0$ un sepeartore in $R^d$, definiamo $a' = (a_1,...,a_d, a_0)$ un separatore in $R^{d+1}$ con offset $0$. Dato un qualsiasi punto $x = (x_1,...,x_d)$ in $R^d$, desiniamo $x' = (x_1,...,x_d, 1)$.

$$
a'\cdot x' = a_1 x_1+...+a_n x_d + a_0 = a\cdot x + a_0
$$

L'iperpiano $(a, a_0)$ separa correttamente i punti $x$ in $R^d$ se e solo se $(a', 0)$ separa correttamente i punti $x'$ in $R^{d+1}$ con ultima coordinata uguale a $1$.

Dal punto di vista dell'algoritmo perceptron

![perceptron no a0](./perceptron_no_a0.png)

### Margine

$$
\frac{a^Tx+a_0}{||a||}
$$

è la distanza di $x$ dall'iperpiano $a,a_0$. Se $x$ è un punto del training set e $y$ la sua etichetta allora

$$
y\frac{a^Tx+a_0}{||a||}
$$

è positivo se e solo se $x$ è classificato correttamente dall'iperpiano. Il *margine* del dataset $D$ è

$$
\min_i\left( y^{(i)}\frac{a^Tx^{(i)}+a_0}{||a||} \right)
$$

Il margine è positivo se e solo se tutti i punti sono classificati correttamente.

### Dimostrazione

Dimostreremo il risutato assumendo offset $0$, ovvero il separatore passa per l'origine. Sia $a^*$ il separatore e $a^{(k)}$ l'iperpiano definito al passo $k$ dell'algoritmo. Dimostreremo che l'algolo tra $a^*$ e $a^{(k)}$ decresce con l'aumentare di $k$.

![beta_angle](./beta_angle.png)

- Sia $\gamma > 0$ il margine di $D$.
- Sia $U > 0$ un limite superiore alla norma di tutti gli $x^{(i)}$, ovvero $||x^{(i)}|| \leq U$.

![proof](./perceptron_proof.png)

# Esercitazione 1

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random as rn

def plot_separation(X, y, a, a0, name=None):
    '''
    Input: X vettore 2xn dove la colonna j rappresenta il punto j nello spazio delle features
        y: vettore di dimensione n, y[j] è l'etichetta di X[:,j:j+1] (colonna j di X)
        a: vettore 2x1
        a0: float, rappresentano i coefficienti del piano di separazione
    Output: None, crea un file png che mostra lo spazio delle features e il piano di separazione
    '''
    plt.scatter(X[0:1,:], X[1:2,:], c=['g' if lab == 1 else 'r' for lab in y], s=7)
     
    ax = plt.gca()
    
    # piano di separazione
    xlim_left, xlim_right = ax.get_xlim()
    ylim_bottom, ylim_top = ax.get_ylim()
    
    plt.plot( ( xlim_left, xlim_right) , [ (-x*a[0][0]-a0)/a[1][0] for x in  (xlim_left, xlim_right) ],\
             linewidth=1,
             c = 'b', zorder=0 ) # a0 è una matrice di dimensione 1x1
     
    arrow_size = 0.05
   
   # assi 
    plt.arrow( xlim_left, 0, xlim_right-xlim_left, 0,   width=0.01,\
              head_width= arrow_size, length_includes_head=True,\
              color='lightgrey', zorder=-1)
    plt.arrow( 0, ylim_bottom, 0, ylim_top-ylim_bottom,   width=0.01,\
              head_width= arrow_size, length_includes_head=True,\
              color='lightgrey', zorder=-1)    


    # plot del vettore perpendicolare al piano
    
    # punto centrale del piano
    mx = (xlim_left+xlim_right)/2
    my = (-mx*a[0][0]-a0)/a[1][0]

    
    
    u = a/np.linalg.norm(a) # vettore unitario ortogonale ad a
    
    # spostiamo l'origine del vettore a sul punto (mx, my) 
    plt.arrow(mx, my, u[0][0], u[1][0], width=0.01,\
              head_width= arrow_size, length_includes_head=True,\
              color='orange', zorder=0)
    
        
    
    ax.set_aspect('equal', adjustable='box')
    
    ax.set_xbound(xlim_left, xlim_right)
    ax.set_ybound(ylim_bottom, ylim_top)
    
    if name != None:
        plt.savefig(name, dpi=600)

    plt.show()

def perceptron( X, y, t = 100 ):
    '''
    Parameters
    ----------
    X : vettore (ndarray) d x n dove d è lo spazio delle features, n numero degli esempi
    y : vettore delle etichette, di dimensione
    t : intero positivo, numero massimo di iterazioni

    Returns
    -------
    a : vettore dei cefficienti dell'iperpiano di dimensione d
    a0: termine noto
    '''
    
    d, n = X.shape
    
    a = np.zeros( (d, 1) )
    a0 = 0
    
    for j in range(t):
        finito = True
        for i in range(n):
            x = X[:,i:i+1] # colonna i di X
            if y[i]*(a.T.dot(x) + a0) <= 0:
                a = a + x*y[i]
                a0 = a0 + y[i]
                finito = False
        if finito: # equivalente a finito == True
            break
                
    return a, a0

## Esercizio 1

Scrivere una funzione, denominata `sign`, che preda in input un iperpiano `h` descritto dal vettore dei coefficienti `a` in $d \times 1$ e termine noto `a0` e un vettore `x` $d \times 1$. Ritorna `1` se  `ax+a0 >= 0`,  `-1` altrimenti

In [8]:
def sign(a, a0, x):
    '''
    Parameters
    ----------
    a : coefficienti, ndarray shape (dx1)
    a0 : termine noto
    x : ndarray shape (dx1)

    Returns
    -------
    int: 1 se ax+a0 > 0 -1 altrimenti

    '''
    # a e x sono matrici, dot è il prodotto tra matrici
    # quindi a.T.dot(x) è una matrice di dimensione 1x1
    # e il suo unico elemento è il prodotto scalare tra a e x
    
    return 1 if a.T.dot(x)+a0 >= 0 else -1

a = np.array( [[2.6], [-1], [3.2]] )
a0 = 5
x = np.array( [ [1.3], [2.1], [0.4]]  )
print(sign(a, a0, x))

1


## Esercizio 2

Modificare la funzione `perceptron()` in modo che ritorni anche il margine del dataset rispetto l'iperpiano trovato nel caso in cui questo lo separi linearmente oppure `None`.

In [11]:
def perceptron( X, y, t = 100):
    '''
    Parameters
    ----------
    X : vettore (ndarray) d x n dove d è lo spazio delle features, n numero degli esempi
    y : vettore delle etichette, di dimensione
    t : intero positivo, numero massimo di iterazioni

    Returns
    -------
    a : vettore dei cefficienti dell'iperpiano di dimensione d
    a0: termine noto
    '''
    
    d, n = X.shape
    
    a = np.zeros( (d, 1) )
    a0 = 0
    
    margine = None
        
    for j in range(t):
        finito = True
 
        for i in range(n):
            x = X[:,i:i+1] # colonna i di X
            if y[i]*(a.T.dot(x) + a0) <= 0:
                a = a + x*y[i]
                a0 = a0 + y[i]
                finito = False
                
            dist = y[i]*(a.T.dot(x) + a0)/ (a**2).sum()**0.5 # matrice 1x1
            dist = dist[0][0]
            try:
                margine = min(margine, dist )
            except TypeError:
                margine = dist
            # oppure ...
            #if margine != None:
            #    margine = min(margine, dist )
            #else:
            #    margine = dist
        if finito:
            break
 
    # in caso di non convergenza margine è negativo
    if margine < 0:
        margine = None
            
    return a, a0, margine

X = [  [1, -1 ], [0.5, 1], [-.6, -.7], [-.8, 0.5]  ]
X = np.array(X).T
y1 = np.array([1, -1, 1, -1])
y2 = np.array([-1, 1, 1, -1])

a, a0, m  = perceptron(X, y1, 100)
print(a, a0, m)

a, a0, m  = perceptron(X, y2, 100)
print(a, a0, m)

[[ 0.5]
 [-2. ]] 0 0.5335783750799324
[[ 0.7]
 [-0.1]] 0 None


## Esercizio 3

Scrivere una funzione, denominata `get_dataset`, che prenda in input un iperpiano `a`, `a0` (`a` è un vettore colonna $d\times 1$ ), un intero `n` ed un float `s`. La funzione ritorni un dataset casuale di `n` punti in $(-s,s)^d$ linearmente separabile da `a`, `a0`.

In particolare la funzione deve ritornare un vettore $d\times n$ dove ogni colonna rappresenta un punto in $(-s,s)^d$ ed un vettore di etichette `y` di dimensione `n` tale che `y[i]` vale `-1` o `1`.

Testare l'algoritmo perceptron modificato con il dataset ritornato dalla funzione, verificare che il margine sia non `None`.

*Suggerimento*. Potrebbero essere utili le funzioni `rn.choice()` per la scelta del segno e `rn.random()` che ritorna un numero causale in $[0,1)$.

## Esercizio 4

Scrivere una funzione, chiamata `flip_labels` che perturbi il dataset fornito dalla  precedente funzione invertendo, con una probabilità p, i valori delle etichette 

La funzione prenda in input il vettore delle etichette `y` di dimensione $1 \times d$, la probabilità `p` e ritorni il nuovo vettore delle etichette perturbato.

Cosa può essere utile: la funzione random.random() restituisce un numero pseudo-casuale
in [0,1)